## Predicting Employees Future

## Ingesting Data

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
 
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler

In [0]:
# File location and type
file_location = "s3://employeeleavingornot/Employee.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.show()

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
Education|JoiningYear| City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
Bachelors| 2017|Bangalore| 3| 34| Male| No| 0| 0|
Bachelors| 2013| Pune| 1| 28|Female| No| 3| 1|
Bachelors| 2014|New Delhi| 3| 38|Female| No| 2| 0|
 Masters| 2016|Bangalore| 3| 27| Male| No| 5| 1|
 Masters| 2017| Pune| 3| 24| Male| Yes| 2| 1|
Bachelors| 2016|Bangalore| 3| 22| Male| No| 0| 0|
Bachelors| 2015|New Delhi| 3| 38| Male| No| 0| 0|
Bachelors| 2016|Bangalore| 3| 34|Female| No| 2| 1|
Bachelors| 2016| Pune| 3| 23| Male| No| 1| 0|
 Masters| 2017|New Delhi| 2| 37| Male| No| 2| 0|
 Masters| 2012|Bangalore| 3| 27| Male| No| 5| 1|
Bachelors| 2016| Pune| 3| 34| Male| No| 3| 0|
Bachelors| 2018| Pune| 3| 32| Male| Yes| 5| 1|
Bachelors| 2016|Bangalore| 3| 39| Male| No| 2| 0|
Bachelors| 2012|Bangalore| 3| 37| Male| No| 4| 0|
Bachelors| 2017|Bangalore| 1| 29| Male| No| 3| 0|
Bachelors| 2014|Bangalore| 3| 34|Female| No| 2| 0|
Bachelors| 2014| Pune| 3| 34| Male| No| 4| 0|
Bachelors| 2015| Pune| 2| 30|Female| No| 0| 1|
Bachelors| 2016|New Delhi| 2| 22|Female| No| 0| 1|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
only showing top 20 rows

In [0]:
df.printSchema()

root
-- Education: string (nullable = true)
-- JoiningYear: integer (nullable = true)
-- City: string (nullable = true)
-- PaymentTier: integer (nullable = true)
-- Age: integer (nullable = true)
-- Gender: string (nullable = true)
-- EverBenched: string (nullable = true)
-- ExperienceInCurrentDomain: integer (nullable = true)
-- LeaveOrNot: integer (nullable = true)

In [0]:
df.count()

Out[75]: 4653

In [0]:
df.columns

Out[76]: ['Education',
 'JoiningYear',
 'City',
 'PaymentTier',
 'Age',
 'Gender',
 'EverBenched',
 'ExperienceInCurrentDomain',
 'LeaveOrNot']

In [0]:
df_Columns=['Education', 'JoiningYear', 'City', 'PaymentTier', 'Age', 'Gender', 'EverBenched', 'ExperienceInCurrentDomain', 'LeaveOrNot']
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]).show()

+---------+-----------+----+-----------+---+------+-----------+-------------------------+----------+
Education|JoiningYear|City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+----+-----------+---+------+-----------+-------------------------+----------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------+-----------+----+-----------+---+------+-----------+-------------------------+----------+

In [0]:
df.show(5)

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
Education|JoiningYear| City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
Bachelors| 2017|Bangalore| 3| 34| Male| No| 0| 0|
Bachelors| 2013| Pune| 1| 28|Female| No| 3| 1|
Bachelors| 2014|New Delhi| 3| 38|Female| No| 2| 0|
 Masters| 2016|Bangalore| 3| 27| Male| No| 5| 1|
 Masters| 2017| Pune| 3| 24| Male| Yes| 2| 1|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
only showing top 5 rows

In [0]:
df.select("Education").distinct().show()

+---------+
Education|
+---------+
 Masters|
Bachelors|
 PHD|
+---------+

In [0]:
df.select("City").distinct().show()

+---------+
 City|
+---------+
Bangalore|
 Pune|
New Delhi|
+---------+

In [0]:
df.select("ExperienceInCurrentDomain").distinct().orderBy("ExperienceInCurrentDomain").show()

+-------------------------+
ExperienceInCurrentDomain|
+-------------------------+
 0|
 1|
 2|
 3|
 4|
 5|
 6|
 7|
+-------------------------+

## Splitting Data

In [0]:
(train_data,test_data) = df.randomSplit([0.7,0.3],42)
 
print("Count of Training records: " + str(train_data.count()))
print("Count of Training records: " + str(test_data.count()))

Count of Training records: 3317
Count of Training records: 1336

In [0]:
categorical_columns = ['Education', 'City', 'Gender', 'EverBenched']

## Building Machine Learning Pipeline

In [0]:
stages = []
#iterating each columns for the specified categorical columns
for catcol in categorical_columns:
    #passing the categorical column into the string indexer for indexing the string values into a numerical values
    stringIndexer = StringIndexer(inputCol=catcol,outputCol=catcol + "Index")
    #taking the output of the string indexer and passing it into the one hot encoder as input
    #the created categorical vector has n columns for the n types of categorical values within each column
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],outputCols=[catcol +"Category_vectors"])
    #stages is used for the pipeline structure creation, no transformation or fitting is done
    stages += [stringIndexer,encoder]

In [0]:
stages

Out[81]: [StringIndexer_64b31559660c,
 OneHotEncoder_0be3f024a70f,
 StringIndexer_f61404ac0079,
 OneHotEncoder_dc2923a45e95,
 StringIndexer_8f9228b14285,
 OneHotEncoder_d5ec7f5535ac,
 StringIndexer_45731a21a6b5,
 OneHotEncoder_eddfb5b40e3d]

In [0]:
numerical_columns = ['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain']

In [0]:
#bringing the categorical columns and numerical columns togeter to be passed into vector assembler
assembleInputs = [c + "Category_vectors" for c in categorical_columns ] + numerical_columns
#vector assembler takes all the values and create an array which is the inputs for the models
assembler = VectorAssembler(inputCols=assembleInputs,outputCol='features')
stages += [assembler]

In [0]:
stages

Out[84]: [StringIndexer_64b31559660c,
 OneHotEncoder_0be3f024a70f,
 StringIndexer_f61404ac0079,
 OneHotEncoder_dc2923a45e95,
 StringIndexer_8f9228b14285,
 OneHotEncoder_d5ec7f5535ac,
 StringIndexer_45731a21a6b5,
 OneHotEncoder_eddfb5b40e3d,
 VectorAssembler_71c8ffec7994]

In [0]:
pipeline = Pipeline().setStages(stages)
pipelineModel = pipeline.fit(train_data)

In [0]:
model =pipeline.fit(train_data)
pipelineModel = pipeline.fit(train_data) 

## Training and Testing Model

In [0]:
trainprepDF = pipelineModel.transform(train_data)
testprepDF = pipelineModel.transform(test_data)

##Logistic Regression

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator

lr = LogisticRegression(featuresCol="features",labelCol='LeaveOrNot')
lrModel=lr.fit(trainprepDF)
lrModelpredictions = lrModel.transform(testprepDF)
evaluator = RegressionEvaluator(
    labelCol="LeaveOrNot", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lrModelpredictions)
model_accuracy =lrModel.summary.accuracy
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.535422

## Decision Tree Regressor

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

dtr = DecisionTreeRegressor(featuresCol="features",labelCol='LeaveOrNot')
dtrModel=dtr.fit(trainprepDF)
dtrModelpredictions = dtrModel.transform(testprepDF)
evaluator = RegressionEvaluator(
    labelCol="LeaveOrNot", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(dtrModelpredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 0.35863

## Random Forest Regressor

In [0]:
from pyspark.ml.regression import RandomForestRegressor
rfr = RandomForestRegressor(featuresCol="features",labelCol='LeaveOrNot')
rfrModel=rfr.fit(trainprepDF)
rfrModelpredictions = rfrModel.transform(testprepDF)
evaluator = RegressionEvaluator(
    labelCol="LeaveOrNot", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfrModelpredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)



Root Mean Squared Error (RMSE) on test data = 0.355921

## Gradient Boosted Tree Regressor

In [0]:
from pyspark.ml.regression import GBTRegressor
GBTR = GBTRegressor(featuresCol="features",labelCol='LeaveOrNot')
GBTRModel=GBTR.fit(trainprepDF)
GBTRModelpredictions = GBTRModel.transform(testprepDF)
evaluator = RegressionEvaluator(
    labelCol="LeaveOrNot", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(GBTRModelpredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.343384

##  Isotonic Regression

In [0]:
from pyspark.ml.regression import IsotonicRegression
IR = IsotonicRegression(featuresCol="features",labelCol='LeaveOrNot')
IRModel=IR.fit(trainprepDF)
IRModelpredictions = IRModel.transform(testprepDF)
evaluator = RegressionEvaluator(
    labelCol="LeaveOrNot", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(IRModelpredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.513978

### Gradient Boosted Tree Regressor is the best performing model with the lowest Root Mean Square Error of 0.343384.